## Functions

In [24]:
from math import acos, pi, sqrt

def sgn(x):
    assert isinstance(x, (int,float)) 
    if x < 0:
        return -1
    if x > 0:
        return 1
    return 0

def _q(M, j, C):
    num = C[M + 2 - j - 1] # 1 based indexing
    denom = sqrt(sum([c*c for c in C[:M + 2 - j]]))
    return num/denom

def compute_theta(M, j, C):
    if j > M:
        raise ValueError
    if j < M:
        return 2*acos(_q(M, j, C))
    return 2*sgn(C[0])*(acos(_q(M, j, C)) - pi) + 2*pi

def compute_all_theta(M, C):
    return [compute_theta(M, j, C) for j in range(1, M+1)]

In [25]:
from math import prod, sqrt

def _w(M, v_a, j, i):
    return 2*M + v_a - 2*j + i

def _x(v_b, j, i):
    return v_b + 2*j + i

# M represents M+1 in the inductive formula
def compute_intermediate_coeff(M, E, v_a, v_b, d_prev, eta=-1):
    amps = dict()
    state_order = []
    for j in range(M):
        d_j = d_prev[j]
        inc_a_state = (_w(M-1, v_a, j, 2), _x(v_b, j, 0))
        # E index should match the current EGO iter (M), but also have 1-indexing
        inc_a_coeff = prod(sqrt(_w(M-1, v_a, j, i+1)) for i in range(2)) / (E[M-1] + eta)
        
        inc_b_state = (_w(M-1, v_a, j, 0), _x(v_b, j, 2))
        inc_b_coeff = prod(sqrt(_x(v_b, j, i+1)) for i in range(2)) / (E[M-1] - eta)

        for state, coeff in [(inc_a_state, inc_a_coeff), (inc_b_state, inc_b_coeff)]:
            if state not in amps:
                state_order.append(state)
                amps[state] = 0
            amps[state] += d_j*coeff
        # print(amps)
        
    
    # assert output length is M+1
    assert len(state_order) == M+1, f'Unexpected state length: {len(state_order)}'

    mag_squared = sum(a*a for a in amps.values())
    mag_squared = 1
    print(state_order)
    return [amps[state]/sqrt(mag_squared) for state in state_order]
        

def compute_ansatz_coeff(M, E, v_a, v_b, eta=-1):
    D = []
    for l in range(1, M+1):
        # in lth iteration, prev output vector is a superposition of l fiducial states
        d_prev = D[-1] if len(D) > 0 else [1]
        #print(f'Norm squared after iter {l-1}: {sum(d_j * d_j for d_j in d_prev)}')
        D.append(compute_intermediate_coeff(l, E, v_a, v_b, d_prev, eta=eta))

    # return D[-1]
    m_sq = sum(d*d for d in D[-1])
    return [d / sqrt(m_sq) for d in D[-1]]

In [26]:
from bethe_functions import Bethe_Equations 
from scipy.optimize import root

def get_starting_params(N):
    # output list of inputs where 2*M + v_a + v_b = N
    assert type(N) is int
    if N % 2 == 1:
        M = N//2
        return [(M, 0, 1, -1), (M, 1, 0, 1)], Bethe_Equations.bethe_func_N_odd 

    return [(N//2, 0, 0, 0), (N//2-1, 1, 1, 1)], Bethe_Equations.bethe_func_N_even

better_init = lambda M: [i/M for i in range(M)]
def get_bethe_ansatz_angles(N, V, init_strat=better_init, method="hybr"):
    params, E_func = get_starting_params(N)
    C_options = []
    angle_options = []
    for M, v_a, v_b, E_v in params:
        E = root(E_func, init_strat(M), args=(M, (V,E_v)), method=method).x
        C = compute_ansatz_coeff(M, E, v_a, v_b)
        C_options.append(C)

        curr_angles = compute_all_theta(M, C)
        angle_options.append(curr_angles)
        print(curr_angles)
        
    return angle_options, C_options

## Tests

In [27]:
THRESHOLD = 1e-5

class Test_N_7:
    N = 7
    W = .5
    V = .75
    E = [1.94591, 1.33363, .701066]
    M = 3
    v_a = 1
    v_b = 0
    eta = -1*sqrt( (V+W) / (V-W) )
    expected_C = [3.40577e-3, -3.08911e-2, .18121, -.982953] # (1,6), (3,4), (5,2), (7,0); or greatest k to least k
    expected_theta = [3.13478, 3.20338, 9.78939]
    
    def test_ansatz_coeff():
        M, E, v_a, v_b = Test_N_7.M, Test_N_7.E, Test_N_7.v_a, Test_N_7.v_b
        expected_C = list(reversed(Test_N_7.expected_C))

        eta = Test_N_7.eta
        print('Eta:', eta)
        C = compute_ansatz_coeff(M, E, v_a, v_b, eta=eta)
        print('Actual:', C)
        print('Expected:', expected_C)
        diffs = [c - expected_c for c, expected_c in zip(C, expected_C)]
        print('Differences:', diffs)
        if all(abs(d) < THRESHOLD for d in diffs):
            print('Good!')
        else:
            print('Inaccurate output!')

    def test_ansatz_angles():
        M, C, expected_theta = Test_N_7.M, Test_N_7.expected_C, Test_N_7.expected_theta
        theta = compute_all_theta(M, list(reversed(C)))
        print('Actual:', theta)
        print('Expected:', expected_theta)
        diffs = [t - expected_t for t, expected_t in zip(theta, expected_theta)]
        print('Differences:', diffs)
        if all(abs(d) < THRESHOLD for d in diffs):
            print('Good!')
        else:
            print('Inaccurate output!')

Test_N_7.test_ansatz_coeff()
print()
print()
Test_N_7.test_ansatz_angles()

Eta: -2.23606797749979
[(3, 0), (1, 2)]
[(5, 0), (3, 2), (1, 4)]
[(7, 0), (5, 2), (3, 4), (1, 6)]
Actual: [-0.9829533553589793, 0.18120943345573517, -0.030890832794998286, 0.003405710558334186]
Expected: [-0.982953, 0.18121, -0.0308911, 0.00340577]
Differences: [-3.5535897935368155e-07, -5.665442648350449e-07, 2.6720500171523165e-07, -5.9441665813900896e-08]
Good!


Actual: [3.1347810987991855, 3.2033850570703435, 9.789389512110574]
Expected: [3.13478, 3.20338, 9.78939]
Differences: [1.0987991854172208e-06, 5.0570703433727715e-06, -4.878894248605548e-07]
Good!


## Computing Eigenvalues

In [28]:
C_1 = compute_ansatz_coeff(M=Test_N_7.M, E=Test_N_7.E, v_a=1, v_b=0, eta=Test_N_7.eta)
coeffs_1 = [(0, C_1[0]), (C_1[1], C_1[1]), (C_1[2], C_1[2]), (C_1[3], 0)]
states_1 = [(7, 0), (5, 2), (3, 4), (1, 6)]
C_1

[(3, 0), (1, 2)]
[(5, 0), (3, 2), (1, 4)]
[(7, 0), (5, 2), (3, 4), (1, 6)]


[-0.9829533553589793,
 0.18120943345573517,
 -0.030890832794998286,
 0.003405710558334186]

In [29]:
E_2 = list(reversed([0.85445148, 1.55778674, 2.14114304]))
#E_2 = Test_N_7.E
C_2 = compute_ansatz_coeff(M=Test_N_7.M, E=E_2, v_a=0, v_b=1, eta=Test_N_7.eta)
coeffs_2 = [(0, C_2[0]), (C_2[1], C_2[1]), (C_2[2], C_2[2]), (C_2[3], 0)]
states_2 = [(6, 1), (4, 3), (2, 5), (0, 7)]
C_2

[(2, 1), (0, 3)]
[(4, 1), (2, 3), (0, 5)]
[(6, 1), (4, 3), (2, 5), (0, 7)]


[-0.9592391431542702,
 0.27777635075069607,
 -0.051780449025694904,
 0.004398897928502951]

In [30]:
def mul_ham_state(states, rel_coeffs, V, W, N):
    out = 0
    for coeffs, state in zip(states, rel_coeffs):
        coeff1, coeff2 = coeffs
        out += (state[1] - state[0])/2
        out += (W/N) * ( (state[1] + state[0])/2 + state[1]*state[0] )
        out += (V/(2*N)) * ( coeff1 + coeff2 )
    return out


In [31]:
mul_ham_state(states_1, coeffs_1, V=.75, W=.5, N=7)

0.9849873156545239

In [32]:
mul_ham_state(states_2, coeffs_2, V=.75, W=.5, N=7)

1.0059250291050272